In [2]:
import numpy as np

In [7]:
A = np.array([[1, 2024],[0,1]])
U, S, Vh = np.linalg.svd(A)

In [8]:
U

array([[ 9.99999878e-01, -4.94070965e-04],
       [ 4.94070965e-04,  9.99999878e-01]])

In [9]:
S

array([2.02400049e+03, 4.94071026e-04])

In [10]:
Vh

array([[ 4.94070965e-04,  9.99999878e-01],
       [-9.99999878e-01,  4.94070965e-04]])